In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datetime import datetime

# 데이터 분석 입문자를 위한 파이썬 판다스 300제
https://wikidocs.net/89378  

# DataManim - 빅데이터 분석기사 실기 문제 연습
https://www.datamanim.com/dataset/99_pandas/pandasMain.html

# 5. 데이터 프레임에 함수 적용하기

## 5-1. lambda 함수
- 이름없이 간단한 함수를 만드는 것 

In [59]:
df = pd.DataFrame({'a': [1, 2, 3, 4, 5], 'b': [4, 5, 6, 7, 8], 'c': [10, 20, 30, 40, 50]})
df

,a,b,c
0,1,4,10
1,2,5,20
2,3,6,30
3,4,7,40
4,5,8,50


In [3]:
def change_a(x):
    return x+1

In [6]:
df['a'].map(lambda x: x+1)

0    False
1     True
2    False
3    False
4    False
Name: a, dtype: bool

## 5-2. map() / apply() / applymap() 비교
- map() : 단일 컬럼(Series)에 적용
- apply() : DataFrame의 행 or 열에 적용
- applymap() : 개별 값에 적용

In [38]:
df['d'] = df['a'].map(lambda x: x+1)

In [39]:
test = {1: '일', 2: '이'}
df['a'].map(test)

0      일
1      이
2    NaN
3    NaN
4    NaN
Name: a, dtype: object

In [48]:
df['sum'] = df.apply(lambda x: x.sum(), axis=1)

In [50]:
df.loc[0, 'a'] = np.nan
df.loc[3, 'b'] = np.nan

In [57]:
df[['a', 'b']] = df[['a', 'b']].apply(lambda x: x.fillna(value=x.mean()))

In [52]:
df['a'].fillna(value=df['a'].mean())
df['b'].fillna(value=df['b'].mean())

0    3.5
1    2.0
2    3.0
3    4.0
4    5.0
Name: a, dtype: float64

In [62]:
df.applymap(lambda x: x*10)

,a,b,c
0,10,40,100
1,20,50,200
2,30,60,300
3,40,70,400
4,50,80,500


# 연습문제
- dataset/titanic.csv 사용

In [65]:
df = pd.read_csv('dataset/titanic.csv')

In [66]:
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


## Q1. 성별을 'male'은 0으로, 'female'은 1로 변환하여 새로운 열 'gender_num'로 추가하세요.

In [69]:
gender_map = {'male': 0, 'female': 1}
df['gender_num'] = df['gender'].map(gender_map)

In [72]:
df['gender'].map(lambda x: 0 if x=='male' else 1)
np.where(df['gender'] == 'male', 0, 1)

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    1
889    0
890    0
Name: gender, Length: 891, dtype: int64

In [71]:
df['gender'].unique()

array(['male', 'female'], dtype=object)

## Q2. 탑승 항구 embarked를 'S'는 'Southampton', 'C'는 'Cherbourg', 'Q'는 'Queenstown'으로 매핑하여 새로운 열 'embarked_port'로 추가하세요.

In [74]:
emb_map = {'S': 'Southampton', 'C': 'Cherbourg', 'Q': 'Queenstown'}
df['embarked_port'] = df['embarked'].map(emb_map)

In [76]:
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,gender_num,embarked_port
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,Southampton


In [77]:
def get_emp_port(x):
    if x == 'S':
        return 'Southampton'
    elif x == 'C':
        return 'Cherbourg'
    elif x == 'Q':
        return 'Queenstown'
    else:
        return None

In [78]:
df['embarked'].map(get_emp_port)

0      Southampton
1        Cherbourg
2      Southampton
3      Southampton
4      Southampton
          ...     
886    Southampton
887    Southampton
888    Southampton
889      Cherbourg
890     Queenstown
Name: embarked, Length: 891, dtype: object

## Q3. 이름의 길이가 20자 이상이면 '긴 이름', 10자 이상이면 '중간 이름', 그 외에는 '짧은 이름'을 반환하는 함수를 작성하고, 이를 'Name_Length_Class' 열에 적용하세요.

In [79]:
def get_length(name):
    if len(name) >= 20:
        return '긴 이름'
    elif len(name) >= 10:
        return '중간 이름'
    else:
        return '짧은 이름'

In [81]:
df['Name_Length_Class'] = df['name'].map(get_length)

In [82]:
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,gender_num,embarked_port,Name_Length_Class
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,Southampton,긴 이름


## Q4. 선실 등급(Pclass)이 1인 승객의 운임 요금(fare)을 1.5배로 계산하고,  선실 등급이 2인 승객의 운임 요금은 해당 승객의 운임 요금을 1.2배로,  선실 등급이 3인 승객의 운임 요금은 그대로 유지하여  조정된 운임요금 열 'adj_fare'를 추가하세요.

In [87]:
df['pclass'].unique()

array([3, 1, 2], dtype=int64)

In [88]:
def get_adj_fare(x):
    if x['pclass'] == 1:
        return x['fare'] * 1.5
    elif x['pclass'] == 2:
        return x['fare'] * 1.2
    else:
        return x['fare']

In [92]:
df['adj_fare'] = df[['pclass', 'fare']].apply(lambda x: get_adj_fare(x), axis=1)

In [94]:
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,gender_num,embarked_port,Name_Length_Class,adj_fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,Southampton,긴 이름,7.25


# 6. 날짜 데이터 처리
- 'dataset/AirQualityUCI.csv' 데이터 사용
- T: 온도
- RH: 상대습도
- AH: 절대 습도

In [95]:
df = pd.read_csv('dataset/AirQualityUCI.csv')

In [105]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [106]:
df.dtypes

datetime    datetime64[ns]
T                  float64
RH                 float64
AH                 float64
dtype: object

## 6-1. 필요한 정보 추출하기
- dt.date
- dt.time
---
- dt.year
- dt.month
- dt.day
- dt.hour
- dt.minute
- dt.second
---
- dt.weekday : The day of the week with Monday=0, Sunday=6.
- datetime.strptime(x, '%Y-%m-%d').strftime('%w') : Sunday=0, Saturday=6

In [110]:
df['datetime'].dt.month

0       3
1       3
2       3
3       3
4       3
       ..
9352    4
9353    4
9354    4
9355    4
9356    4
Name: datetime, Length: 9357, dtype: int64

## datetime.datetime
- 날짜와 시간(datetime)을 문자열로 출력하려면 strftime
- 날짜와 시간 형식의 문자열을 datetime으로 변환하려면 strptime

## lv2 기출 
### map,  datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%w') 활용해서 'weekday' 컬럼 추가

In [114]:
df['datetime'] = df['datetime'].astype(str)

In [115]:
df.dtypes

datetime     object
T           float64
RH          float64
AH          float64
dtype: object

In [119]:
df['weekday'] = df['datetime']\
    .map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%w'))

In [122]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [123]:
df['weekday_dt'] = df['datetime'].dt.weekday

In [124]:
df.head(1)

,datetime,T,RH,AH,weekday,weekday_dt
0,2004-03-10 18:00:00,13.6,48.875001,0.757754,3,2


# DD/MM/YYYY

In [131]:
df_test = pd.DataFrame({'date': ['01/01/2024',
                                 '01/02/2024',
                                 '01/03/2024',
                                 '01/04/2024',
                                 '01/05/2024']})

In [134]:
pd.to_datetime(df_test['date'], format='%d/%m/%Y')

0   2024-01-01
1   2024-02-01
2   2024-03-01
3   2024-04-01
4   2024-05-01
Name: date, dtype: datetime64[ns]

In [136]:
df[df['datetime'] < '2005-01-01']

,datetime,T,RH,AH,weekday,weekday_dt
0,2004-03-10 18:00:00,13.600,48.875001,0.757754,3,2
1,2004-03-10 19:00:00,13.300,47.700000,0.725487,3,2
2,2004-03-10 20:00:00,11.900,53.975000,0.750239,3,2
3,2004-03-10 21:00:00,11.000,60.000000,0.786713,3,2
4,2004-03-10 22:00:00,11.150,59.575001,0.788794,3,2
...,...,...,...,...,...,...
7105,2004-12-31 19:00:00,9.350,33.625000,0.395774,5,4
7106,2004-12-31 20:00:00,9.625,32.300000,0.387114,5,4
7107,2004-12-31 21:00:00,9.125,32.475000,0.376616,5,4
7108,2004-12-31 22:00:00,9.100,37.225000,0.430991,5,4


# 7. 데이터 집계와 그룹연산
- groupby
- crosstab
- pivot_table

In [137]:
df = pd.read_csv('dataset/titanic.csv')
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [144]:
df.groupby(['pclass', 'survived'])['fare'].agg(['mean', 'sum'])

mean         sum
pclass survived                       
1      0         64.684008   5174.7206
       1         95.608029  13002.6919
2      0         19.412328   1882.9958
       1         22.055700   1918.8459
3      0         13.669364   5085.0035
       1         13.694887   1629.6916

## 7-1. groupby

## Q. 클래스(pclass)와 생존여부(survived)별 운임(fare)의 합을 구하세요.

In [149]:
df.groupby(['pclass', 'survived'])['fare'].sum().reset_index()

,pclass,survived,fare
0,1,0,5174.7206
1,1,1,13002.6919
2,2,0,1882.9958
3,2,1,1918.8459
4,3,0,5085.0035
5,3,1,1629.6916


## 7-2. crosstab
## 성별(gender)과 생존 여부(survived)에 대한 교차표

In [155]:
pd.crosstab(index=df['gender'], columns=df['survived'], normalize='columns')

survived,0,1
gender,,
female,0.147541,0.681287
male,0.852459,0.318713


## 7-3. pivot_table
## 클래스(pclass)와 생존여부(survived)별 운임(fare)의 평균을 구하세요.

In [157]:
pd.pivot_table(df,
               index='pclass',
               columns='survived',
               values='fare',
               aggfunc='mean')

survived,0,1
pclass,,
1,64.684008,95.608029
2,19.412328,22.055700
3,13.669364,13.694887


In [159]:
df.groupby(['pclass', 'survived'])['fare'].agg('mean').unstack('survived')

survived,0,1
pclass,,
1,64.684008,95.608029
2,19.412328,22.055700
3,13.669364,13.694887


## 연습문제
- dataset/diamonds.csv 사용

| 컬럼명      | 설명                                                         |
|-------------|--------------------------------------------------------------|
| carat       | 다이아몬드의 무게(carat)                                      |
| cut         | 다이아몬드의 귀빛(cut) 등급                                  |
| color       | 다이아몬드의 색상(color) 등급                                 |
| clarity     | 다이아몬드의 투명도(clarity) 등급                             |
| depth       | 다이아몬드의 심도(depth) 비율                                 |
| table       | 다이아몬드의 상단 테이블(table) 너비 비율                    |
| price       | 다이아몬드의 가격(price)                                      |
| x           | 다이아몬드의 가로 길이(x)                                     |
| y           | 다이아몬드의 세로 길이(y)                                     |
| z           | 다이아몬드의 깊이(z)                                           |
---
- groupby
- crosstab
- pivot_table
---

In [160]:
df = pd.read_csv('dataset/diamonds.csv')
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


# Q1.  다이아몬드의 cut 등급에 따른 평균 가격 계산하기

In [169]:
df.groupby('cut')['price'].mean().to_frame()
pd.pivot_table(df, index='cut', values='price', aggfunc='mean')

,price
cut,
Fair,2793.612903
Good,2185.303371
Ideal,2503.051051
Premium,2554.372414
Very Good,2365.309735


## Q2. 다이아몬드의 color 등급과 clarity 등급에 따른 빈도표 만들기

In [177]:
df.groupby(['color', 'clarity'])['cut'].count().unstack('clarity')

clarity,I1,IF,SI1,SI2,VS1,VS2,VVS1,VVS2
color,,,,,,,,
D,1.0,NaN,60.0,22.0,9.0,23.0,7.0,7.0
E,4.0,3.0,87.0,25.0,33.0,65.0,9.0,14.0
F,7.0,3.0,44.0,33.0,58.0,59.0,13.0,9.0
G,3.0,6.0,28.0,20.0,26.0,34.0,5.0,17.0
H,6.0,1.0,39.0,27.0,15.0,15.0,14.0,8.0
I,6.0,1.0,30.0,16.0,13.0,13.0,10.0,6.0
J,2.0,NaN,18.0,11.0,5.0,9.0,NaN,1.0


In [172]:
df.groupby(['color', 'clarity']).size().unstack('clarity')

clarity,I1,IF,SI1,SI2,VS1,VS2,VVS1,VVS2
color,,,,,,,,
D,1.0,NaN,60.0,22.0,9.0,23.0,7.0,7.0
E,4.0,3.0,87.0,25.0,33.0,65.0,9.0,14.0
F,7.0,3.0,44.0,33.0,58.0,59.0,13.0,9.0
G,3.0,6.0,28.0,20.0,26.0,34.0,5.0,17.0
H,6.0,1.0,39.0,27.0,15.0,15.0,14.0,8.0
I,6.0,1.0,30.0,16.0,13.0,13.0,10.0,6.0
J,2.0,NaN,18.0,11.0,5.0,9.0,NaN,1.0


In [170]:
pd.crosstab(index=df['color'], columns=df['clarity'])

clarity,I1,IF,SI1,SI2,VS1,VS2,VVS1,VVS2
color,,,,,,,,
D,1,0,60,22,9,23,7,7
E,4,3,87,25,33,65,9,14
F,7,3,44,33,58,59,13,9
G,3,6,28,20,26,34,5,17
H,6,1,39,27,15,15,14,8
I,6,1,30,16,13,13,10,6
J,2,0,18,11,5,9,0,1


## Q3. 다이아몬드의 cut 등급에 따른 carat 평균과 가격 중간값 계산하기

In [178]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [185]:
df.groupby('cut').apply(lambda x: pd.Series({'carat': x['carat'].mean(),
                                             'price': x['price'].median()}))
df.groupby('cut').agg({'carat': 'mean', 'price': 'median'})

,carat,price
cut,,
Fair,0.900806,2843.5
Good,0.645281,2801.0
Ideal,0.666306,2822.0
Premium,0.709828,2822.0
Very Good,0.656062,2809.5


# 8. 그룹별 함수적용

## 8-1. apply()

In [219]:
df = pd.read_csv('dataset/titanic.csv')

In [220]:
df = df[['gender', 'fare']].head()

In [221]:
df

,gender,fare
0,male,7.2500
1,female,71.2833
2,female,7.9250
3,female,53.1000
4,male,8.0500


In [197]:
mean_fare = df.groupby('gender')['fare'].mean().reset_index()
mean_fare

,gender,fare
0,female,44.102767
1,male,7.650000


In [199]:
pd.merge(df, mean_fare, on='gender', how='left', suffixes=('', '_mean'))

,gender,fare,fare_mean
0,male,7.2500,7.650000
1,female,71.2833,44.102767
2,female,7.9250,44.102767
3,female,53.1000,44.102767
4,male,8.0500,7.650000


## 8-2. transform()

In [218]:
df

,gender,fare
0,male,7.2500
1,female,71.2833
2,female,7.9250
3,female,53.1000
4,male,8.0500


In [206]:
df['fare_mean'] = df.groupby('gender')['fare'].transform(lambda x: x.mean())

In [222]:
df.iloc[3, -1] = np.nan

In [223]:
df

,gender,fare
0,male,7.2500
1,female,71.2833
2,female,7.9250
3,female,NaN
4,male,8.0500


In [228]:
df['fare'] = df.groupby('gender')['fare'].\
                transform(lambda x: x.fillna(value=x.mean()))

In [229]:
df

,gender,fare
0,male,7.25000
1,female,71.28330
2,female,7.92500
3,female,39.60415
4,male,8.05000


# 1. age의 결측치를 pclass별 age 중간값으로 대체하세요.

In [242]:
df = pd.read_csv('dataset/titanic.csv')

In [236]:
df['age'].isna().sum()

177

In [237]:
df['age'] = df.groupby('pclass')['age'].transform(lambda x: x.fillna(value=x.median()))

In [238]:
df['age'].isna().sum()

0

# 2. pclass 탑승자수를 new_column으로 정의하세요.(기존 데이터프레임 인덱스 유지) 탑승자 수는 고유한 passengerId를 기준으로 한다

In [243]:
df['new_column'] = df.groupby('pclass')['passengerId'].transform(lambda x: x.nunique())

In [245]:
df.head()

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,new_column
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,491
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,216
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,491
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,216
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,491


# 연습문제
- dataset/titanic.csv 활용

## Q1. titanic 데이터셋에서 각 승객 등급('pclass')별로 생존한 승객 수를 계산하고, 이를 원래 데이터프레임에 새로운 열 'SurvivedCountByClass'로 추가하세요.

In [248]:
df['SurvivedCountByClass'] = df.groupby('pclass')['survived']\
                                .transform(lambda x: x.sum())

In [254]:
survived = df.groupby('pclass')['survived'].sum()
survived

pclass
1    136
2     87
3    119
Name: survived, dtype: int64

In [255]:
df.head(1)

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,new_column,SurvivedCountByClass
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,491,119


In [258]:
pd.merge(df, survived, left_on='pclass', right_index=True, how='left',
         suffixes=('', '_y')).rename(columns={'survived_y':'SurvivedCountByClass2'})

,passengerId,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked,new_column,SurvivedCountByClass,SurvivedCountByClass2
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,491,119,119
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,216,136,136
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,491,119,119
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,216,136,136
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,491,119,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,184,87,87
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,216,136,136
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,491,119,119
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,216,136,136


## Q2. titanic 데이터셋에서 age의 결측치를 그 승객의 성별의 평균나이로 대체하세요.